In [7]:
%load_ext autoreload

In [21]:
%autoreload 2
import sys

sys.path.append("../")
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator
from models.Ground_truth_oracles.TF_binding_landscape_models import *
from explorers.bo_explorer import BO_Explorer

LANDSCAPE_TYPES_PROTEIN = {"RNA": [], "TF": [], "Protein": ["PF1"]}
bo_explorer_prod = BO_Explorer(virtual_screen=20)
bo_explorer_prod.debug = False
evaluator_bo = Evaluator(
    bo_explorer_prod,
    landscape_types=LANDSCAPE_TYPES_PROTEIN,
    path="../simulations/Protein/eval_BO/",
)
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=3)


loading landscapes RNA: [], TF:[], Protein:['PF1']
0 TF landscapes loaded.
core.import_pose.import_pose: {0} File '../data/Protein_landscapes/folding_landscapes/pdb3msh.ent' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue ALA:CtermProteinFull 90
1 Protein landscapes loaded.
loading complete
Running on PF1
start seq PF1
Evaluating for virtual_screen: 1
Running  NAM {'landscape_id': 'PF1', 'start_id': 'PF1', 'signal_strength': 1}


RuntimeError: Caught an unknown exception!